# BB84 Quantum Key Distribution Protocol

This notebook implements the BB84 quantum key distribution protocol using Qiskit. It simulates the process of Alice preparing and sending qubits, Bob measuring them, and both parties comparing bases to extract a shared key. This implementation assumes a noise-free quantum channel.


In [175]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator


In [176]:
# Number of qubits (key length before sifting)
num_qubit = 64


## Step 1: Alice generates her random bits and basis

- Alice generates a completely random sequence of bits (0 or 1).
- These bits are not a message — they are just a temporary random sequence for key generation.
- She also randomly chooses a basis (Z or X) for each bit:
  - **Z basis** (computational): encodes 0 as |0⟩ and 1 as |1⟩.
  - **X basis** (Hadamard): encodes 0 as |+⟩ and 1 as |−⟩.
- These bits and bases will be used to prepare the quantum states she sends to Bob.


In [177]:
# Alice's random bit values (0 or 1)
A_bits = np.random.randint(0, 2, num_qubit)

# Alice's random basis choices (0 = Z basis, 1 = X basis)
A_base = np.random.randint(0, 2, num_qubit)


## Step 2: Alice prepares the qubits based on bits and basis

- Alice encodes each bit using the basis she selected:
  - If the bit is 1, she applies an X gate.
  - If the basis is X, she applies a Hadamard gate.
- The resulting qubits are now in superposition states and ready to be sent.


In [178]:
def Alice_preparation(bits, bases, circuit):
    circuit.barrier()
    
    # Step 1: Encode the bits
    # If the bit is 1, apply an X gate to flip |0⟩ to |1⟩
    for i in range(len(bits)):
        if bits[i] == 1:
            circuit.x(i)
    
    circuit.barrier()
    
    # Step 2: Apply basis encoding
    # If the basis is X (i.e., 1), apply a Hadamard gate to move to the |+⟩ / |−⟩ basis
    for i in range(len(bases)):
        if bases[i] == 1:
            circuit.h(i)
    
    circuit.barrier()



## Step 3: Bob measures the qubits

- Bob independently chooses a random basis (Z or X) for each qubit he receives from Alice.
- If his chosen basis is **X**, he applies a Hadamard gate to rotate the qubit into the computational basis.
- He then performs a standard measurement in the computational (Z) basis.
- The results of these measurements form Bob's raw bit sequence.



In [179]:
# Bob's random basis choices for each qubit
B_base = np.random.randint(0, 2, num_qubit)

def Bob_measurement(bases, circuit):
    # Step 1: Apply basis
    # If Bob's basis is X (1), apply a Hadamard to rotate to the computational basis
    for i in range(len(bases)):
        if bases[i] == 1:
            circuit.h(i)
    
    # Step 2: Measure all qubits in the computational (Z) basis
    circuit.measure_all()


## Step 4: Simulating the quantum transmission

- In a real BB84 protocol, Alice sends qubits through a quantum channel to Bob.
- In our simulation, both Alice’s preparation and Bob’s measurement are applied in sequence within a single quantum circuit.
- We use Qiskit’s `AerSimulator` to simulate the execution of the circuit.
- The circuit includes:
  - Alice’s preparation of the qubits.
  - Bob’s basis choices and measurements.
- The output from the simulator mimics what Bob would measure in real life.


In [180]:
# Initialize the circuit
qc = QuantumCircuit(num_qubit)

# Add the Alice preparation
Alice_preparation(A_bits, A_base, qc)

# Add the Bob measurement
Bob_measurement(B_base, qc)

# Print the circuit (Optional)
#display(qc.draw('mpl'))

# Simulate the circuit
simulator = AerSimulator()
result = simulator.run(qc, shots=1, memory=True).result()

# Extract the bitstring from the result
bitstring = result.get_memory()[0].replace(" ", "")[::-1]
B_bits = np.array([int(bit) for bit in bitstring])


## Step 5: Extracting the shared key

- After the simulation, Alice and Bob compare the bases they used.
- They keep only the bits where their basis choices matched — this process is called **key sifting**.
- If no noise or eavesdropping occurred, these shared bits should be identical.
- This forms the basis of their secret key.


In [181]:
# Indices where bases matched
matching = (A_base == B_base)

A_shared = A_bits[matching]
B_shared = B_bits[matching]

print("Alice's shared key:", A_shared)
print("Bob's shared key:  ", B_shared)


Alice's shared key: [1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0]
Bob's shared key:   [1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0]


## Step 6: Comparing results

- We compare Alice’s and Bob’s shared keys to see if they agree.
- The percentage of matching bits gives us the **key agreement rate**.
- In a perfect simulation without noise, this rate should be 100%.
- In a real system, a lower rate may indicate:
  - Noise in the quantum channel.
  - An eavesdropper (Eve) interfering with the transmission.


In [182]:
matches = A_shared == B_shared
agreement_rate = np.sum(matches) / len(matches)

print(f"Key agreement rate: {agreement_rate:.2%}")


Key agreement rate: 100.00%


## Optional: Simulating a Noisy Quantum Channel

- In the real world, the quantum channel between Alice and Bob is not perfect — qubits can be affected by decoherence, loss, or unwanted interactions.
- To simulate this, we apply **depolarizing noise** to each qubit **after Alice's preparation and before Bob's measurement**.
- Depolarizing noise randomly replaces a qubit’s state with a completely mixed state with a given probability.
- This models a noisy transmission line while keeping Alice’s and Bob’s operations ideal.
- The noise level can be adjusted to observe how increasing imperfections impact the agreement rate between Alice and Bob’s shared keys.


In [183]:
from qiskit_aer.noise import depolarizing_error

def apply_channel_noise(qc, noise_level) -> None:
    # Create a depolarizing noise channel with the given error probability
    error_channel = depolarizing_error(noise_level, 1).to_instruction()

    # Apply the noise channel to each qubit to simulate transmission noise
    for i in range(qc.num_qubits):
        qc.append(error_channel, [i])


In [184]:
# Create a new quantum circuit for the noisy BB84 protocol
qc_noisy = QuantumCircuit(num_qubit)

# Alice encodes her bits using her chosen bases
Alice_preparation(A_bits, A_base, qc_noisy)

# Apply depolarizing noise to simulate a noisy quantum channel
apply_channel_noise(qc_noisy, noise_level=0.5)

# Bob measures the received qubits using his own random bases
Bob_measurement(B_base, qc_noisy)

# Simulate the full noisy circuit
simulator = AerSimulator()
result_noisy = simulator.run(qc_noisy, shots=1, memory=True).result()

# Extract Bob's measurement results and convert to array
bitstring_noisy = result_noisy.get_memory()[0].replace(" ", "")[::-1]
B_bits_noisy = np.array([int(b) for b in bitstring_noisy])

# Keep only the bits where Alice and Bob used the same basis
matching_noisy = (A_base == B_base)
A_shared_noisy = A_bits[matching_noisy]
B_shared_noisy = B_bits_noisy[matching_noisy]

# Print shared key results and agreement rate
print("Alice's shared key (noisy):", A_shared_noisy)
print("Bob's shared key   (noisy):", B_shared_noisy)

matches_noisy = A_shared_noisy == B_shared_noisy
agreement_rate_noisy = np.sum(matches_noisy) / len(matches_noisy)
print(f"Key agreement rate (noisy): {agreement_rate_noisy:.2%}")


Alice's shared key (noisy): [1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0]
Bob's shared key   (noisy): [1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0]
Key agreement rate (noisy): 65.38%


Information reconciliation can be applied to the noisy shared key to improve the agreement rate between Alice and Bob. However, this process typically reduces the final key length, as some bits must be discarded or publicly verified.

An eavesdropper (Eve) can also be introduced by intercepting the qubits sent by Alice, measuring them in a random basis, and then preparing and forwarding new qubits to Bob. Due to the no-cloning theorem, Eve's interference disturbs the quantum states and introduces detectable errors. By publicly comparing a small, randomly chosen subset of their bits, Alice and Bob can estimate the error rate and detect the presence of an eavesdropper.

## References

- [1] Wikipedia, *Quantum Key Distribution*.  
  https://en.wikipedia.org/wiki/Quantum_key_distribution

- [2] Nielsen, M. A., & Chuang, I. L. (2000). *Quantum Computation and Quantum Information*. Cambridge University Press.  

- [3] IBM Qiskit Documentation. 
  https://qiskit.org/learn/

